In [1]:
import requests, json, os, warnings, re
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import sys
import glob

#### Loading testing database

#### Loading training database

In [10]:
data_raw= pd.read_json("./1.TrainingData/all.json") 

In [11]:
data_raw[0:3]

,Exclude,appId,comment,dataSource,date,fee,future,id,label,lemmatized_comment,...,reviewId,reviewer,sentiScore,sentiScore_neg,sentiScore_pos,stemmed,stopwords_removal,stopwords_removal_lemmatization,stopwords_removal_nltk,title
0,NaN,None,"Besides the occasional crash, this is an amazi...",RE2014_app_and_play_store_apps,None,None,0,264,Bug,"besides the occasional crash, this be an amaze...",...,10946,None,3,-1,3,"besid the occa crash, thi is an amaz produc wi...","besides occasional crash, this amazing product...","besides occasional crash, this amaze product w...","besides occasional crash, amazing product tons...",Almost perfect
1,NaN,382698565,This could be a great app if it was predictabl...,AppStore_Random,None,None,0,111,Bug,this could be a great app if it be predictable...,...,638,None,3,-1,3,"thi could be a gre ap if it was predictable, b...","this could be great app if was predictable, bu...","this could be great app if be predictable, but...","could great app predictable, full bugs unpredi...",Take a photo of your boarding pass
2,NaN,None,I can't open since the last 2 updates Pop-ups ...,RE2014_app_and_play_store_apps,None,None,0,210,Bug,i can't open since the last 2 update pop-up go...,...,3129,None,2,-1,2,i can't op sint the last 2 upd pop-ups go craz...,can't open since last 2 updates pop-ups go cra...,can't open since last 2 update pop-up go crazy...,can't open since last 2 updates pop-ups go cra...,Won't open due to pop-ups since upgrade


In [43]:
trainingText = data_raw["stopwords_removal_nltk"]
trainingResponse=data_raw.label

#### 1.  STARTING TO TRAIN THE MODEL

#### 1.1 Vectorizing the data

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=1000)
trainingText_stopwords_removal_nlt = vect.fit_transform(trainingText).toarray()
for i in range(5):
    print(sum(trainingText_stopwords_removal_nlt[i]))

5
16
8
14
55


#### 1.2 Training the model

In [51]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(trainingText_stopwords_removal_nlt, trainingResponse)

# Predict Class
y_pred = classifier.predict(trainingText_stopwords_removal_nlt)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(trainingResponse, y_pred)
accuracy

0.25602817664589544

#### 2. STARTING TESTING MODEL

#### 2.1 Loading Testing Data 

In [54]:
testingData=pd.read_csv("./2.TestingData/Data542_finaldata_final.csv")

In [55]:
testingData.head()

,Unnamed: 0,index,appTitle,userName,date,score,text,category,appTitle_lower,text_org,contentRating,Review Length,month
0,0,2,Photomath,NaN,"March 8, 2019",5,Great App Detailed answers Understandable answers,EDUCATION,photomath,Great App. Detailed answers. Understandable an...,Everyone,49,3
1,1,6,Photomath,אוביידו אדריאנה,"March 8, 2019",5,Great tool for learning math I love it,EDUCATION,photomath,"Great tool for learning math, I love it!",Everyone,38,3
2,2,9,Photomath,Kaleb Wallis,"March 8, 2019",5,great if your in school or have homework,EDUCATION,photomath,great if your in school or have homework,Everyone,40,3
3,3,10,Photomath,juana,"March 8, 2019",4,It is working amazing for me Now i love to fin...,EDUCATION,photomath,It is working amazing for me!!! Now i love to ...,Everyone,159,3
4,4,11,Photomath,Duje Basic,"March 8, 2019",5,really great and you have all the explanations...,EDUCATION,photomath,really great and you have all the explanations...,Everyone,72,3


#### 2.1 Preprocessing the testing data

In [ ]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 


testingText = testingData.text

#Preprocessing the data: Remove none alphabetic characters/Make the word lower case/Remove the stop words

def mutlti_func(x):       
    text= word_tokenize(str(re.sub('[^A-Za-z]', ' ', x)).lower())
    for word in text:
        if word in stopwords.words('english'):
            text.remove(word)
    return text


testingText=testingText.apply(mutlti_func)
testingText.head()

In [ ]:
from nltk.stem.porter import PorterStemmer
from spellchecker import SpellChecker


# Stemming / Spelling 
spell = SpellChecker()
stemmer = PorterStemmer()

def stem_spell_fuc(x):
    for i in range(len(x)):
        #x[i] = stemmer.stem(spell.correction(x[i]))
        x[i] = stemmer.stem(x[i])
    return x

testingText=testingText.apply(stem_spell_fuc)
testingText.head()

In [ ]:
def join_fuc(x):
    x = " ".join(x)
    return x

testingText=testingText.apply(join_fuc)
testingText

#### 2.2 Running testing data

In [ ]:
# Predict Class
y_testing_pred = classifier.predict(testingText)
y_testing_pred.head(20)
